В этом ноутбуке берется переписанный код статьи и проверяется корректность его запуска, а именно, что итоговые метрики будут похожи на предоставленные в статье.

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm.tabddpm import *

### Подготовка

In [3]:
model_name = 'tabddpm'
dataname = 'beijing'
model_short = 'TabDDPM'
save_cat = None

In [4]:
CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('method', model_name)
CONFIG.add_arg('save_cat', save_cat)

In [5]:
download_dataset(dataname)
process_data(dataname)

Start processing dataset beijing from UCI.
Aready downloaded.
{'cat_col_idx': [0, 1, 2, 3, 8],
 'column_names': None,
 'data_path': 'data/beijing/beijing.csv',
 'file_type': 'csv',
 'header': 'infer',
 'name': 'beijing',
 'num_col_idx': [5, 6, 7, 9, 10, 11],
 'raw_data_path': 'data/beijing/PRSA_data_2010.1.1-2014.12.31.csv',
 'target_col_idx': [4],
 'task_type': 'regression',
 'test_path': None}
n_clusters: 25
beijing (37581, 12) (4176, 12) (41757, 12)


/workspace/TabDDPM_copy/TabDDPM/utils.py:373: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.rename(columns = idx_name_mapping, inplace=True)
/workspace/TabDDPM_copy/TabDDPM/utils.py:374: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.rename(columns = idx_name_mapping, inplace=True)
/workspace/TabDDPM_copy/TabDDPM/utils.py:379: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_df.loc[train_df[col] == '?', col] = 'nan'
/workspace

Numerical (37581, 6)
Categorical (37581, 5)
Processing and Saving beijing Successfully!
beijing
Total 41757
Train 37581
Test 4176
Num 7
Cat 5


In [18]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")


CONFIG.add_arg('sample_save_path',
               f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")
CONFIG.add_arg('real_path',
               f"synthetic/{CONFIG.get_arg('dataname')}/real.csv")
CONFIG.add_arg('test_path',
               f"synthetic/{CONFIG.get_arg('dataname')}/test.csv")
CONFIG.add_arg('info_path',
               f"data/{CONFIG.get_arg('dataname')}/info.json")

Device: cuda


In [16]:
CONFIG.get_all_args()

{'dataname': 'beijing',
 'method': 'tabddpm',
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': 'synthetic/beijing/tabddpm.csv',
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 66,
 'real_path': 'synthetic/beijing/real.csv',
 'test_path': 'synthetic/beijing/test.csv',
 'info_path': 'data/beijing/info.json',
 'save_path': './synthetic/shoppers_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_mult_9.csv',
 'num_clusters': 25,
 'save_cat': None}

### Train

In [8]:
CONFIG.add_arg('mode', 'train')

In [9]:
tabddpm = TabDDPM(CONFIG)

In [ ]:
tabddpm.train()

START TRAINING
No NaNs in numerical features, skipping
83
{'num_classes': 2, 'is_y_cond': False, 'rtdl_params': {'d_layers': [1024, 2048, 2048, 1024], 'dropout': 0.0}, 'd_in': 83}
mlp
MLPDiffusion(
  (mlp): MLP(
    (blocks): ModuleList(
      (0): Block(
        (linear): Linear(in_features=1024, out_features=1024, bias=True)
        (activation): ReLU()
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (1): Block(
        (linear): Linear(in_features=1024, out_features=2048, bias=True)
        (activation): ReLU()
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (2): Block(
        (linear): Linear(in_features=2048, out_features=2048, bias=True)
        (activation): ReLU()
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (3): Block(
        (linear): Linear(in_features=2048, out_features=1024, bias=True)
        (activation): ReLU()
        (dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (head): Linear(in_features=1024, out_feature

In [ ]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)
    
cmap = get_cmap(10)

In [ ]:
loss = pd.read_csv(f'./models/{model_name}/ckpt/{dataname}/loss.csv')
max_v, min_v = -np.inf, np.inf
window = 250

start_step = 100
for i, l in enumerate(loss.columns[1:]):
    plt.plot(loss['step'][start_step:], loss[l][start_step:], alpha=0.25, color=cmap(i))
    smoothed = np.convolve(loss[l][start_step:], np.ones(window)/window, 'valid')  
    plt.plot(loss['step'][start_step+window-1:], smoothed, label=l, color=cmap(i))

    max_v = np.max([np.quantile(loss[l][start_step:], 0.99), max_v])
    min_v = np.min([np.quantile(loss[l][start_step:], 0.01), min_v])

plt.ylim([min_v-0.1, max_v+0.1]) 
plt.legend()
plt.title(f"Model name: {model_name.upper()}\nLoss by steps from step #{start_step}")
plt.savefig(f'./models/{model_name}/ckpt/{dataname}/loss_image.png')
plt.show()

In [ ]:
tabddpm.sample(sample_save_path=CONFIG.get_arg('sample_save_path'))

### Расчет метрик

In [9]:
from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

In [10]:
overall_metrics = {}
overall_metrics[model_name] = {}

In [11]:
CONFIG.get_all_args()

{'dataname': 'beijing',
 'method': 'tabddpm',
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': 'synthetic/beijing/tabddpm.csv',
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 66,
 'real_path': 'synthetic/beijing/real.csv',
 'test_path': 'synthetic/beijing/test.csv',
 'info_path': 'data/beijing/info.json',
 'save_path': './synthetic/shoppers_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_mult_9.csv',
 'num_clusters': 25,
 'save_cat': None}

In [13]:
pd.read_csv(CONFIG.get_arg('sample_save_path'))

,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,2013,11,30,1,145.703610,-14.000000,-1.0,1028.0000,cv,0.450000,0.0,0.0
1,2014,7,7,20,78.000000,21.000000,31.0,1002.0000,SE,21.910000,0.0,0.0
2,2013,10,16,10,68.414580,4.000000,15.0,1017.0000,cv,1.790000,0.0,0.0
3,2014,2,21,17,504.721470,-3.000000,6.0,1028.0000,cv,1.790000,0.0,0.0
4,2013,10,24,11,24.092388,-5.000000,14.0,1035.0000,NW,10.280000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
37576,2014,10,4,4,62.000000,12.000000,15.0,1017.7278,NE,1.790000,0.0,0.0
37577,2010,5,14,12,10.000000,-2.758127,26.0,1010.0000,NW,2.680000,0.0,0.0
37578,2010,11,23,15,235.505400,-2.000000,9.0,1014.0000,SE,1.790000,0.0,0.0
37579,2011,1,27,23,29.000000,-18.000000,-5.0,1041.6565,cv,3.444402,0.0,0.0


In [14]:
pd.read_csv(CONFIG.get_arg('real_path'))

,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,2012,3,24,23,45.0,-11.0,4.0,1022.0,cv,0.89,0.0,0.0
1,2012,6,6,7,148.0,14.0,23.0,1002.0,cv,2.23,0.0,0.0
2,2012,3,30,23,37.0,-11.0,10.0,1017.0,SE,3.13,0.0,0.0
3,2011,1,6,21,21.0,-19.0,-6.0,1037.0,NW,422.42,0.0,0.0
4,2014,10,16,20,55.0,-1.0,13.0,1018.0,SE,53.64,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
37576,2013,3,13,15,12.0,-19.0,8.0,1028.0,cv,2.68,0.0,0.0
37577,2013,6,15,5,70.0,14.0,20.0,1008.0,SE,1.79,0.0,0.0
37578,2011,1,14,10,20.0,-20.0,-4.0,1035.0,NW,73.76,0.0,0.0
37579,2012,8,27,15,44.0,22.0,29.0,1008.0,SE,10.29,0.0,0.0


#### Метрики из статьи TabSyn

In [12]:
overall_metrics[model_name]['similarity'] = calculate_similarity()

CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 12/12 [00:00<00:00, 38.47it/s]|
Column Shapes Score: 99.05%

(2/2) Evaluating Column Pair Trends: |██████████| 66/66 [00:00<00:00, 66.94it/s]|
Column Pair Trends Score: 97.47%

Overall Score (Average): 98.26%

Error rate (%) of column-wise density estimation TABDDPM: 0.950 ± 0.380
Error rate (%) of pair-wise column correlation score TABDDPM: 2.533 ± 1.930


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


#### MLE

In [19]:
overall_metrics[model_name]['mle'] = calculate_mle()

100%|██████████| 36/36 [00:23<00:00,  1.52it/s]


Saving scores to  eval/mle/beijing/tabddpm.json
RMSE обучения на синтетических данных TABDDPM: 0.635 ± 0.044


#### Detection: classifier two sample tests (C2ST)

In [20]:
overall_metrics[model_name]['detection'] = calculate_detection()

beijing, tabddpm: 0.977025902489196


#### DCR

In [21]:
overall_metrics[model_name]['DCR'] = calculate_DCR()

n_clusters: 25
DCR Score, a value closer to 0.5 is better
beijing-tabddpm, DCR Score = 0.5071711769245097


#### Alpha precision & Beta recall

In [22]:
overall_metrics[model_name]['quality'] = calculate_alpha_beta()

=========== All Features ===========
Data shape:  (37581, 83)
alpha precision: 0.990344, beta recall: 0.541752


### Табличка

In [23]:
final_metrics_table = []
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':'TabDDPM', 'Type':m, 'Data':dataname}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [24]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(4)

similarity  \
     Model     Type     Data Column Shapes Score, %   
0  TabDDPM  tabddpm  beijing                99.0503   

                                                          \
  Column Pair Trends Score, % Overall Score (Average), %   
0                     97.4674                    98.2588   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                              0.9497   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                  0.3803   

                                                           \
  Error rate (%) of pair-wise column correlation score, %   
0                                                  2.5326   

                                                               \
  Error rate (%) of pair-wise column correlation score std, %   
0                                                      1.9302   

                                    mle  \
  RMSE обучения на синтетических данных   
0                                0.6349   

                                             detection     DCR  \
  RMSE обучения на синтетических данных, std     Score   Score   
0                                      0.044     0.977  0.5072   

          quality              
  alpha precision beta recall  
0          0.9903      0.5418

In [25]:
import os
if not os.path.exists(f'./eval/total/{dataname}'):
    os.makedirs(f'./eval/total/{dataname}')
final_metrics_table.to_csv(f'./eval/total/{dataname}/{model_short}_final_metrics_table.csv', index=False)